In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!ln -s /content/drive/My\ Drive/ /my\drive

ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [18]:
#파이토치 불러오기
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [19]:
#버전 낮춰서 pillow install 하기 (pytorch 버전과 맞추기 위함이다)

!pip install "pillow<6.2.2"

In [20]:
!pip install torchvision

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [22]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [30]:
EPOCHS = 50
BATCH_SIZE = 64

In [31]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                    train=False,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))                          
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

In [37]:
class Net(nn.Module):
  def __init__(self, dropout_p=0.2):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)

    # 드롭아웃 확률
    self.dropout_p = dropout_p

  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))

    # 드롭아웃 추가
    x = F.dropout(x, training=self.training, p=self.dropout_p)
    x = F.relu(self.fc2(x))
    
    # 드롭아웃 추가
    x = F.dropout(x, training=self.training, p=self.dropout_p)
    x = self.fc3(x)
    return x

In [38]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [39]:
def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

In [40]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)
      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      # 맞춘 갯수 계산
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. *correct /len(test_loader.dataset)
  return test_loss, test_accuracy

In [41]:
for epoch in range(1, EPOCHS + 1):
  train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)

  print('[{}], Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1], Test Loss: 0.5469, Accuracy: 82.59%
[2], Test Loss: 0.4246, Accuracy: 86.57%
[3], Test Loss: 0.3482, Accuracy: 89.12%
[4], Test Loss: 0.2943, Accuracy: 90.98%
[5], Test Loss: 0.2566, Accuracy: 92.16%
[6], Test Loss: 0.2301, Accuracy: 92.83%
[7], Test Loss: 0.2074, Accuracy: 93.52%
[8], Test Loss: 0.1878, Accuracy: 94.16%
[9], Test Loss: 0.1763, Accuracy: 94.61%
[10], Test Loss: 0.1660, Accuracy: 94.95%
[11], Test Loss: 0.1575, Accuracy: 95.25%
[12], Test Loss: 0.1532, Accuracy: 95.43%
[13], Test Loss: 0.1450, Accuracy: 95.62%
[14], Test Loss: 0.1386, Accuracy: 95.68%
[15], Test Loss: 0.1348, Accuracy: 95.84%
[16], Test Loss: 0.1280, Accuracy: 95.88%
[17], Test Loss: 0.1263, Accuracy: 96.07%
[18], Test Loss: 0.1214, Accuracy: 96.17%
[19], Test Loss: 0.1220, Accuracy: 96.19%
[20], Test Loss: 0.1177, Accuracy: 96.24%
[21], Test Loss: 0.1139, Accuracy: 96.37%
[22], Test Loss: 0.1135, Accuracy: 96.31%
[23], Test Loss: 0.1091, Accuracy: 96.50%
[24], Test Loss: 0.1084, Accuracy: 96.43%
[